# First-order and total Sobol index

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from SALib.sample import saltelli
from SALib.analyze import sobol
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
import fullmodel_v1_7
import pandas as pd
from Extract_data import extractData
from scipy.optimize import curve_fit
import matplotlib.cm as cm
from matplotlib.ticker import MaxNLocator
from datetime import datetime, timedelta
from multiprocess import Pool, cpu_count

In [6]:
def model_Sobol_index(z_m,
                      d_E,
                      z_mK_bg,
                      p_in,
                      P_0,
                      Temp):

    import numpy as np
    import fullmodel_v1_7
    import pandas as pd
    from Extract_data import extractData
    from scipy.optimize import curve_fit
    # Initial conditions
    M_0 = 0
    A_0 = 0.5
    B_0 = 0.004  # 0.004
    Q_B_0 = 0.01
    Q_A_0 = 0.01
    P_0 = P_0  # 10
    D_0 = 0.0239  # 0.0239
    Y_0 = 0.025
    W_0 = 0.02
    O_0 = 7
    v_A_0 = 0
    v_D_0 = 0
    v_Y_0 = 0
    v_W_0 = 0
    
    
    model_CyB = fullmodel_v1_7.modelCyB()
    
    initial_conditions = [M_0, B_0, A_0,
                          Q_B_0,
                          Q_A_0, P_0,
                          D_0,
                          Y_0, W_0,
                          v_A_0, v_D_0,
                          v_Y_0, v_W_0, O_0]

    # Data
    year = "2021"
    data = pd.read_excel("all-bloom-indicators.xlsx")
    
    lake_name = "Pigeon Lake"
    
    data = data[data["Waterbody name"] == lake_name]
    
    data_fit = extractData(data, year)
    
    
    t_data = data_fit["days"].values
    
    
        
    def read_params():
        import fullmodel_v1_7
        model_CyB = fullmodel_v1_7.modelCyB()
        model_CyB.initial = initial_conditions
        name_data_param = "fitting_parameters_full_variables_v1"
        coment = "_v1_7_"
        name_data = './New data/'+lake_name + '/' \
            + name_data_param + year + lake_name+coment \
            + "_final" + ".csv"
    
        params_fit = pd.read_csv(name_data)
    
        # params_fit = pd.read_csv(
        #     "fitting_parameters_full_variables_v12021Pine Lake_v2__final.csv")
        unknow_params = ["e_BD", "alpha_B", "alpha_D",
                         "alpha_Y",
                         "tau_B", "tau_D", "tau_Y",
                         "a_A", "a_D", "sigma_A",
                         "sigma_D", "x_A", "x_D"]
    
        model_CyB.initial[1] = params_fit["B_0"][0]
        model_CyB.initial[2] = params_fit["A_0"][0]
        model_CyB.initial[6] = params_fit["D_0"][0]
    
        for name in unknow_params:
            model_CyB.params[name] = params_fit[name][0]
    
        # Death Daphnia
        model_CyB.params['n_D'] = 0.0623
        model_CyB.params['e_BD'] = 0.8
        model_CyB.params["tau_B"] = 1.23
        model_CyB.params["alpha_B"] = 0.0035
        # model_CyB.params["tau_Y"] = 1.23
        # # model_CyB.params["tau_D"] = 0.01
        # model_CyB.params["alpha_D"] = 0.003941
        # model_CyB.params['phi_D'] = 700
        # model_CyB.params['phi_Y'] = 700
    
        model_CyB.params['r_Y'] = 2
        model_CyB.params['r_W'] = 1
        model_CyB.params['Ext_Y'] = 0.025*13
        model_CyB.params['Ext_W'] = 0.025
        # model_CyB.params['p_in'] = 0.15*2
    
        # With Toxine
        model_CyB.toxines = True and (model_CyB.initial[1] > 0)
        # New time scale
        # days_before = 15
    
        # days_after = 60
    
        model_CyB.t_0 = 0
    
        # model_CyB.t_f = t_data.max()-t_data.min() + days_after
    
        model_CyB.t_f = 365
    
        model_CyB.delta_t = model_CyB.t_f*3
    
        model_CyB.set_linetime()
    
        t_data = data_fit["days"].values
    
        # Temperature
    
        model_CyB.max_temp_time = 197.3989-20
        # model_CyB.max_temp = 25.9
        # model_CyB.freq_temp = -0.0172
        # model_CyB.init_temp = t_data.min()
    
        # New time
        # t_data = t_data-t_data.min() + days_before
        return model_CyB


    def Change_Temp(model_CyB, Temp):
        import copy
        from scipy.optimize import curve_fit
        import numpy as np
        if Temp != 0:
            # Data Temperature
            days_temp = np.array(
                [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334])
            temp_water = np.array(
                [7.35, 7.3, 7.91, 9.4, 11.67, 13.59, 15.49, 16.02, 14.08, 11.29, 9.26, 8.07])
    
            delta_temp = Temp
    
            for i in range(temp_water.shape[0]):
                if temp_water[i] > 11:
                    temp_water[i] = temp_water[i] + delta_temp
    
            # Define the logistic function
            def temperature(t, K, T, t_0, k_0):
                return K*np.exp(-T*(t-t_0)**2)+k_0
    
            # Set bounds for positive values
            bounds = ([0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf])
    
            # Fit the logistic function to the data
            initial_guess = [15, 0, 200, 7]
            params, covariance = curve_fit(temperature,
                                           days_temp,
                                           temp_water,
                                           p0=initial_guess,
                                           bounds=bounds)
    
            model_CyB.max_temp = round(params[0], 5)
            model_CyB.min_temp = round(params[3], 5)
            model_CyB.max_temp_time = round(params[2]-20, 5)
            model_CyB.T = round(params[1], 5)
            return model_CyB
    
    model_CyB = read_params()
    model_CyB.params['z_m'] = z_m

    model_CyB.params['z_mk_A'] = (z_m*0.0004)*1000
    model_CyB.params['z_mk_B'] = (z_m*0.0004)*1000
    model_CyB.params['z_mK_bg'] = (z_m*z_mK_bg)

    model_CyB.params['d_E'] = d_E
    model_CyB.params['z_mK_bg'] = z_mK_bg*z_m
    model_CyB.params['p_in'] = p_in
    model_CyB.initial[5] = P_0
    print(model_CyB.initial[5])

    model_CyB = Change_Temp(model_CyB, Temp)
    solution, info = model_CyB.solver()
    M_values, B_values, A_values, \
        Q_B_values, Q_A_values, P_values, \
        D_values, Y_values, W_values, \
        v_A_values, v_D_values, v_Y_values, \
        v_W_values, O_values = solution.T

    return B_values

In [7]:
problem = {
    'num_vars': 6,
    'names': ['z_m', 'd_E', 'z_mK_bg', 'p_in', 'P_0', 'Temp'],
    'bounds': [[1, 10], 
               [0.02, 0.08], 
               [0.3, 1.0], 
               [0.1, 0.7], 
               [0.1, 0.7], 
               [0, 3.4]]
}
# sample
param_values = saltelli.sample(problem, 2**10)


C:\Users\brian\AppData\Local\Temp\ipykernel_7536\3078115119.py:12: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 2**10)


In [8]:
param_values.shape, (6*2+2)*2**10

((14336, 6), 14336)

In [9]:
# # Evaluate
# with Pool() as p:
#     y = np.array(p.map(model_Sobol_index, param_values))

In [10]:
Yis = []
NUM_PROCESSES = 6
pool = Pool(NUM_PROCESSES)
for i in range(param_values.shape[0]):
    Yi = pool.apply_async(model_Sobol_index, param_values[i,:])
    Yis.append(Yi)

results = np.array([Yi.get() for Yi in Yis])


In [11]:
results.shape

(14336, 1095)

In [12]:
# Saving
df = pd.DataFrame(results)
# Save the DataFrame to a CSV file
# df.to_csv("Sobol_indexV2.csv", header=False, index=False)
# print('File is saved')

File is saved
